In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

Mounted at /gdrive/


In [ ]:
!python --version

Python 3.7.10


In [ ]:
import transformers
print(torch.__version__)

ModuleNotFoundError: ignored

In [ ]:
# !pip3 install fast-bert
!git clone https://github.com/NVIDIA/apex
!pip3 install --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Cloning into 'apex'...
remote: Enumerating objects: 8038, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 8038 (delta 58), reused 68 (delta 29), pack-reused 7913
Receiving objects: 100% (8038/8038), 14.10 MiB | 19.43 MiB/s, done.
Resolving deltas: 100% (5459/5459), done.
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Processing ./apex
Skipping wheel build for apex, due to binaries being disabled for it.
    Running setup.py install for apex ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-req-build-og0wjqhx/setup.py'"'"'; __file__='"'"'/tmp/pip-req-build-og0wjqhx/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f

In [ ]:
import nltk, re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
r_anum = re.compile(r'([^\sa-z.(?)!])+')
r_white = re.compile(r'[\s.(?)!]+')

def preprocess(post):
	
	post = word_tokenize(post)
	post_preproc = [lemmatizer.lemmatize(w) for w in post if not w.strip().lower() in stop_words]
	post = str(' '.join(post)).lower()
	row_clean = r_white.sub(' ', r_anum.sub('', post.lower())).strip()
	return row_clean

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
!pip3 install transformers
!pip3 install simpletransformers

from sklearn.metrics import f1_score, accuracy_score
def macro_f1(y_true, y_pred):
  return f1_score(y_true, y_pred, average='macro')

from simpletransformers.classification import ClassificationModel
import pandas as pd, numpy as np
import logging
from sklearn.model_selection import train_test_split


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.

tdata = pd.read_excel('/gdrive/My Drive/data_2k.xlsx')
# tdata = pd.read_csv('/gdrive/My Drive/data_2k.csv')

train_df, test_df = train_test_split(tdata, test_size=0.15, random_state=5)
train_df, eval_df = train_test_split(train_df, test_size=0.15, random_state=5)

# Create a ClassificationModel
args = {
    "output_dir": "/gdrive/My Drive/honors/new/outputs/",
    "cache_dir": "/gdrive/My Drive/honors/new/cache_dir/",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "num_train_epochs": 10,
    # "learning_rate": 0.00001,
    "save_steps": 6000,
    "overwrite_output_dir": True,
    "reprocess_input_data": True,
    "evaluate_during_training": True,
    "n_gpu": 1,
    "evaluate_during_training_verbose": True,
    "save_model_every_epoch": False
}

print(len(train_df), len(test_df))

     |████████████████████████████████| 2.1MB 5.3MB/s 
     |████████████████████████████████| 3.3MB 35.0MB/s 
     |████████████████████████████████| 901kB 37.4MB/s 
     |████████████████████████████████| 215kB 4.3MB/s 
     |████████████████████████████████| 1.2MB 7.6MB/s 
     |████████████████████████████████| 122kB 16.2MB/s 
     |████████████████████████████████| 8.2MB 13.8MB/s 
     |████████████████████████████████| 2.1MB 41.6MB/s 
     |████████████████████████████████| 225kB 36.2MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 112kB 41.2MB/s 
     |████████████████████████████████| 163kB 40.8MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 4.2MB 18.9MB/s 
     |████████████████████████████████| 102kB 10.5MB/s 
     |████████████████████████████████| 133kB 40.0MB/s 
     |████████████████████████████████| 112kB 37.9MB/s 
 

1559 324


In [ ]:
train_df['label'] = train_df.apply(lambda row: int(row[1]), axis=1)
eval_df['label'] = eval_df.apply(lambda row: int(row[1]), axis=1)
test_df['label'] = test_df.apply(lambda row: int(row[1]), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
model = ClassificationModel('xlnet', 'xlnet-base-cased', args=args, num_labels=2, use_cuda=True) # You can set class weights by using the optional weight argument
model.train_model(train_df, eval_df=test_df, f1=macro_f1, acc=accuracy_score)

INFO:filelock:Lock 140217856554768 acquired on /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346.lock


Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

INFO:filelock:Lock 140217856554768 released on /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346.lock
INFO:filelock:Lock 140217856549008 acquired on /root/.cache/huggingface/transformers/9461853998373b0b2f8ef8011a13b62a2c5f540b2c535ef3ea46ed8a062b16a9.3e214f11a50e9e03eb47535b58522fc3cc11ac67c120a9450f6276de151af987.lock


Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

INFO:filelock:Lock 140217856549008 released on /root/.cache/huggingface/transformers/9461853998373b0b2f8ef8011a13b62a2c5f540b2c535ef3ea46ed8a062b16a9.3e214f11a50e9e03eb47535b58522fc3cc11ac67c120a9450f6276de151af987.lock
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly i

...

Running Epoch 9 of 10:   0%|          | 0/195 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/324 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file /gdrive/My Drive/honors/new/cache_dir/cached_dev_xlnet_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.8951660858806589, 'tp': 98, 'tn': 211, 'fp': 5, 'fn': 10, 'auroc': 0.9879543895747599, 'auprc': 0.9824936683220097, 'f1': 0.9472925049074365, 'acc': 0.9537037037037037, 'eval_loss': 0.3300652381918198}
INFO:simpletransformers.classification.classification_model: Training of xlnet model complete. Saved to /gdrive/My Drive/honors/new/outputs/.


(1950,
 {'acc': [0.9351851851851852,
   0.9104938271604939,
   0.9537037037037037,
   0.9444444444444444,
   0.9475308641975309,
   0.9598765432098766,
   0.9537037037037037,
   0.9598765432098766,
   0.9537037037037037,
   0.9537037037037037],
  'auprc': [0.9627689294369173,
   0.9692036616970215,
   0.9814291326472832,
   0.9825548844759143,
   0.9825181301298626,
   0.9839517680844878,
   0.9841630537847336,
   0.983727237425493,
   0.9832315673601254,
   0.9824936683220097],
  'auroc': [0.9678283607681756,
   0.9784164951989025,
   0.9862611454046639,
   0.9863683127572016,
   0.9879758230452675,
   0.9892403978052127,
   0.989690500685871,
   0.9895404663923183,
   0.9887688614540466,
   0.9879543895747599],
  'eval_loss': [0.26910590049356403,
   0.34407102902669734,
   0.19095319752274761,
   0.2608100678296135,
   0.2622731819737521,
   0.2472218437966459,
   0.3038969130938335,
   0.2737744659307504,
   0.32143215802454683,
   0.3300652381918198],
  'f1': [0.9250784562021692,


In [ ]:
model = ClassificationModel('xlnet', "/gdrive/My Drive/honors/outputs/", args=args, num_labels=2)

404 Client Error: Not Found for url: https://huggingface.co//gdrive/My%20Drive/honors/outputs//resolve/main/config.json


OSError: ignored

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_score, acc=accuracy_score)
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_score, acc=accuracy_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/276 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file /gdrive/My Drive/honors/new/cache_dir/cached_dev_xlnet_128_2_2


Running Evaluation:   0%|          | 0/35 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.9043127148807496, 'tp': 88, 'tn': 176, 'fp': 3, 'fn': 9, 'auroc': 0.9896907216494846, 'auprc': 0.98521112225526, 'f1': 0.9361702127659575, 'acc': 0.9565217391304348, 'eval_loss': 0.29401688736319814}
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/324 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file /gdrive/My Drive/honors/new/cache_dir/cached_dev_xlnet_128_2_2


Running Evaluation:   0%|          | 0/41 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.8951660858806589, 'tp': 98, 'tn': 211, 'fp': 5, 'fn': 10, 'auroc': 0.9879543895747599, 'auprc': 0.9824936683220097, 'f1': 0.9289099526066351, 'acc': 0.9537037037037037, 'eval_loss': 0.3300652381918198}
